In [3]:
import langchain

In [4]:
(langchain.__version__)

'0.0.354'

In [5]:
import fitz
import pytesseract
from PIL import Image
from io import BytesIO
import os

In [6]:
from dotenv import load_dotenv

In [7]:
load_dotenv()

True

In [8]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate

In [9]:
def tesseract_extract_from_pdf(pdf_path: str) -> str:
    full_ocr_text = ""
    doc = fitz.open(pdf_path)

    for i, page in enumerate(doc):
        pix = page.get_pixmap(dpi=300)
        img = Image.open(BytesIO(pix.tobytes("ppm")))
        page_text = pytesseract.image_to_string(img)
        full_ocr_text += page_text + "\n\n"
        print(f"Page {i+1}: {len(page_text.strip())} characters")

    doc.close()
    return full_ocr_text

In [16]:
tesseract_extract_from_pdf(
    r"C:\Users\rajak\Desktop\OCR\OCR\Sale Deed.pdf"
)


Page 1: 2353 characters
Page 2: 2396 characters


'A\nSALE DEED ——_ | wn\nThis DEED OF ABSOLUTE SALE executed at on this the day of , 201X\n\nby s/o residing at\nB\n\nhereinafter called the VENDOR of the one part which expression shall include his executors,\nadministrators, legal representatives, successors etc.\n\nos\n\nw/o residing at\n\nTO AND IN FAVOUR OF\n\nherein after called the PURCHASER of the Other Part which expression wherever the context\nso requires shall mean and include his heirs, executors, administrators, legal representatives,\nsuccessors etc.\n\nWHEREAS the VENDOR herein has purchased the said property more fully described in\n\nthe Schedule hereunder from Thiru. D  inand by sale deed dated E and registeredon F\nas Document No. G of (year) of Book 1 volume No. wliled aly pages to —_— on the\nfile of the Sub Registrar of — || <—————___\n\nWHEREAS the VENDOR herein has been in exclusive possession and enjoyment of the\nproperty more fully described in the Schedule hereunder with a constructed house thereon\n,which w

In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "meta-llama/Llama-3.1-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto"
)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0.0
)

def correct_text_with_llm(raw_ocr_text: str) -> str:
    prompt = f"""
You are an expert legal proofreader.

Fix ONLY OCR spelling and transcription errors.
Do NOT summarize.
Do NOT add content.

OCR TEXT:
---
{raw_ocr_text}
---

CLEAN TEXT:
"""
    output = generator(prompt)
    return output[0]["generated_text"].split("CLEAN TEXT:")[-1].strip()


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct.
401 Client Error. (Request ID: Root=1-693d662a-5f9f9538765920153f602614;13518596-b532-470a-b244-9ec6f6cf0c3b)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.1-8B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.